In [1]:
import numpy as np
import pandas as pd
import torch

from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import model_checkpoint
from pytorch_lightning.core import LightningModule

from torch.utils.data import DataLoader, Dataset

from transformers.optimization import AdamW, get_cosine_schedule_with_warmup
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel

import re
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = '<s>'
EOS = '</s>'
MASK = '<unused0>'
SENT = '<unused1>'
PAD = '<pad>'


In [3]:
koGPT2_TOKENIZER = PreTrainedTokenizerFast.from_pretrained(
    'skt/kogpt2-base-v2', bos_token=BOS, eos_token=EOS, unk_token='<unk>', pad_token=PAD, mask_token=MASK)
model = GPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2')


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'GPT2Tokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.


In [10]:
charbot_data = pd.read_csv('./Utils/FData.csv')
charbot_data.drop(columns=['Unnamed: 0'], inplace=True)
charbot_data['req'] = charbot_data['req'].str.replace("키키", "ㅋㅋ")
charbot_data['res'] = charbot_data['res'].str.replace("키키", "ㅋㅋ")
charbot_data['label'] = 0
charbot_data['req'] = charbot_data['req'].astype(str)
charbot_data['res'] = charbot_data['res'].astype(str)
df = charbot_data[~charbot_data['req'].str.contains(r"\*", regex=True)]
df = charbot_data[~charbot_data['res'].str.contains(r"\*", regex=True)]
charbot_data = df
charbot_data

,req,res,label
0,언니 명랑 핫도그 먹어 봤어?,웅 먹은 지 한참 됐어,0
1,나 오늘 먹었는데 진짜 존맛 ㅜ,나도 오랜만에 먹고 싶다/어디서?,0
2,나 포장해 와서 집에서 먹었음!,누구랑 먹었어/일 안 했어?,0
3,나 퇴근하고 포장해 왔지!,ㅋㅋ 요새 없어진 거 같던데 나도 찾아봐야지,0
4,언니 지금 논산에 있나?,아니/나 대전 온 지 한참이야 ㅡㅡ!,0
...,...,...,...
42096,언니 정도면 우유빛깔 피부야,그래 그 정도면 관리가 잘 되는 편이지,0
42097,나 이제부터 피부에 신경 좀 쓰려고 해,언니 좋은 꿀팁 있으면 공유하자,0
42098,턱선도 자꾸 무너지고 하루가 다르게 늙고 있어,콜라겐을 아무리 먹어도 티도 안나,0
42099,난 마사지 같은 걸 받고 싶어,나두 피부샵 가서 전신마사지 받고 싶어,0


In [5]:
# 파이토치 데이터셋 만들기

class ChatbotDataset(Dataset):
    def __init__(self, chars, max_len=64):  # 데이터셋 전처리
        self._data = chars
        self.max_len = max_len
        self.q_token = Q_TKN
        self.a_token = A_TKN
        self.sent_token = SENT
        self.eos = EOS
        self.mask = MASK
        self.tokenizer = koGPT2_TOKENIZER

    def __len__(self):  # chatbotdata 의 길이를 리턴
        return len(self._data)

    def __getitem__(self, idx):  # 로드한 챗봇 데이터를 차례차례 DataLoader로 넘겨주는 메서드
        turn = self._data.iloc[idx]
        q = turn['req']  # 질문을 가져온다.

        a = turn['res']  # 답변을 가져온다.

        q_token = self.tokenizer.tokenize(
            self.q_token + str(q) + self.sent_token)
        q_len = len(q_token)

        a_token = self.tokenizer.tokenize(self.a_token + str(a) + self.eos)
        a_len = len(a_token)

        # 질문의 길이가 최대길이보다 크면
        if q_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_token = q_token[-(int(self.max_len / 2)):]  # 질문길이를 최대길이의 반으로
                q_len = len(q_token)
                a_len = self.max_len - q_len
            a_token = a_token[:a_len]  # 답변의 길이를 최대길이 - 질문길이
            a_len = len(a_token)

        # 질문의 길이 + 답변의 길이가 최대길이보다 크면
        if q_len + a_len > self.max_len:
            a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            if a_len <= 0:  # 질문의 길이가 너무 길어 질문만으로 최대 길이를 초과 한다면
                q_token = q_token[-(int(self.max_len / 2)):]  # 질문길이를 최대길이의 반으로
                q_len = len(q_token)
                a_len = self.max_len - q_len  # 답변의 길이를 최대길이 - 질문길이
            a_token = a_token[:a_len]
            a_len = len(a_token)

        # 답변 labels = [mask, mask, ..., mask, ..., <bos>, ...답변... <eos>, <pad> ...]
        labels = [self.mask,] * q_len + a_token[1:]

        # mask = 질문길이 0 + 답변길이 1 + 나머지 0
        mask = [0] * q_len + [1] * a_len + [0] * (self.max_len - q_len - a_len)

        # 답변 labels를 index로 만든다.
        labels_ids = self.tokenizer.convert_tokens_to_ids(labels)

        # 최대길이만큼 padding
        while len(labels_ids) < self.max_len:
            labels_ids += [self.tokenizer.pad_token_id]

        # 질문 + 답변 index 로 만든다.
        token_ids = self.tokenizer.convert_tokens_to_ids(q_token + a_token)

        # 최대길이만큼 padding
        while len(token_ids) < self.max_len:
            token_ids += [self.tokenizer.pad_token_id]

        # 질문+답변, 마스크, 답변
        return (token_ids, np.array(mask), labels_ids)


In [6]:
def collate_batch(batch):
    data = [item[0] for item in batch]
    mask = [item[1] for item in batch]
    label = [item[2] for item in batch]
    return torch.LongTensor(data), torch.LongTensor(mask), torch.LongTensor(label)

In [7]:
train_set = ChatbotDataset(charbot_data, max_len=40)

# 윈도우 환경에서 num_workers 는 무조건 0으로 지정, 리눅스에서는 2
train_dataloader = DataLoader(
    train_set, batch_size=32, num_workers=0, shuffle=True, collate_fn=collate_batch)
print(device)

cuda


In [8]:
model.to(device)
model.train()


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [9]:
learning_rate = 3e-5
criterion = torch.nn.CrossEntropyLoss(reduction='none')
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

epoch = 10
Sneg = -1e18


In [ ]:
from tqdm import tqdm
print("start")
for epoch in tqdm(range(epoch)):
    for batch_idx, samples in enumerate(train_dataloader):
        optimizer.zero_grad()
        token_ids, mask, label = samples
        token_ids, mask, label = token_ids.to(
            device), mask.to(device), label.to(device)
        out = model(token_ids)
        out = out.logits  # Returns a new tensor with the logit of the elements of input
        mask_3d = mask.unsqueeze(dim=2).repeat_interleave(
            repeats=out.shape[2], dim=2)
        mask_out = torch.where(mask_3d == 1, out, Sneg * torch.ones_like(out))
        loss = criterion(mask_out.transpose(2, 1), label)
        # 평균 loss 만들기 avg_loss[0] / avg_loss[1] <- loss 정규화
        avg_loss = loss.sum() / mask.sum()
        avg_loss.backward()
        # 학습 끝
        optimizer.step()
print("end")

In [ ]:
torch.save(model.state_dict(), "model.pt")


In [ ]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(
                Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            input_ids = input_ids.to(device)
            pred = model(input_ids)
            pred = pred.logits
            gen = koGPT2_TOKENIZER.convert_ids_to_tokens(
                torch.argmax(pred, dim=-1).cpu().squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))


In [ ]:
def run_model(model_path):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = torch.load(model_path).to(device)
    model.eval()
    with torch.no_grad():
        while 1:
            q = input("user > ").strip()
            if q == "quit":
                break
            a = ""
            while 1:
                input_ids = torch.LongTensor(koGPT2_TOKENIZER.encode(
                    Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
                input_ids = input_ids.to(device)
                pred = model(input_ids)
                pred = pred.logits
                gen = koGPT2_TOKENIZER.convert_ids_to_tokens(
                    torch.argmax(pred, dim=-1).cpu().squeeze().numpy().tolist())[-1]
                if gen == EOS:
                    break
                a += gen.replace("▁", " ")
            print("Chatbot > {}".format(a.strip()))
